测试时增强(TTA)
1. TestTimeAug
2. 模型推理及结果融合

In [ ]:
#1.tta
#为mmcv中TestTimeAug类，配置如下
tta_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(
        type='TestTimeAug',
        transforms=[
            [dict(type='Resize', img_scale=(1333, 800), keep_ratio=True)],
            [dict(type='RandomFlip', flip_ratio=0.),
                dict(type='RandomFlip', flip_ratio=1.)],
            [dict(type='PackXXXInputs', keys=['img'])],
        ])
]

#注意翻转那里有两个配置，一个概率为0一个为1，则输入的图片将变成两张图像
#得到的results结果将被包装成如下,注意是一张图为一个results
#results=dict(
# input=[...],
# data_samples=[...]
# )其中input和data_samples的长度都为2

定义TTA模型融合策略

In [ ]:
#模型的merge_pred方法接受的参数如下：
# data_{i}_{j} 表示对第 i 张图片做第 j 种增强后的结果，
# 例如 batch_size=3，那么 i 的 取值范围为 0，1，2，
# 增强方式有 2 种（翻转），那么 j 的取值范围为 0，1
"""
bs=3
demo_results = [
    [data_0_0, data_0_1],
    [data_1_0, data_1_1],
    [data_2_0, data_2_1],
]
"""
#这个结果和上述的有出入，应该是tta增强后还做了其他操作

In [ ]:
from mmengine.model import BaseTTAModel
from mmengine.structures import BaseDataElement
from typing import List

class ClsDataSample(BaseDataElement):
    pass

class AverageClsScoreTTA(BaseTTAModel):
    def merge_preds(self,data_samples_list:List[List[ClsDataSample]])->List[ClsDataSample]:
        merge_data_samples=[]
        for data_samples in data_samples_list:
            merge_data_sample:ClsDataSample=data_samples[0].new()#第一张图是原图
            merge_score=sum(data_sample.pred_label.score for data_sample in data_samples) / len(data_samples)
            merge_data_sample.set_pred_score=merge_score
            merge_data_samples.append(merge_data_sample)
        return merge_data_samples
    
#使用
tta_model = dict(type='AverageClsScoreTTA')
from mmengine.config import ConfigDict,Config

cfg=Config.fromfile('...')
cfg.model = ConfigDict(**cfg.tta_model, module=cfg.model)
cfg.test_dataloader.dataset.pipeline = cfg.tta_pipeline

![模型关系](./src/2.png) \
![调用堆栈顺序](./src/3.png)


In [ ]:
#数据经过TTA数据增强后
image1  = dict(
    inputs=[data_1_1, data_1_2],
    data_sample=[data_sample1_1, data_sample1_2]
)

image2  = dict(
    inputs=[data_2_1, data_2_2],
    data_sample=[data_sample2_1, data_sample2_2]
)


image3  = dict(
    inputs=[data_3_1, data_3_2],
    data_sample=[data_sample3_1, data_sample3_2]
)
#经过dataloader之后
data_batch = dict(
    inputs = [
              (data_1_1, data_2_1, data_3_1),
              (data_1_2, data_2_2, data_3_2),
             ]
    data_samples=[
         (data_samples1_1, data_samples2_1, data_samples3_1),
         (data_samples1_2, data_samples2_2, data_samples3_2)
     ]
)
#推理之前BaseTTAModel会处理为
data_batch_aug1 = dict(
    inputs = (data_1_1, data_2_1, data_3_1),
    data_samples=(data_samples1_1, data_samples2_1, data_samples3_1)
)

data_batch_aug2 = dict(
    inputs = (data_1_2, data_2_2, data_3_2),
    data_samples=(data_samples1_2, data_samples2_2, data_samples3_2)
)
#推理之后处理为，并送入融合模型
preds = [
    [data_samples1_1, data_samples_1_2],
    [data_samples2_1, data_samples_2_2],
    [data_samples3_1, data_samples_3_2],
]